# gee tools download practice

In [63]:
import ee
import urllib.request
import pandas as pd
import numpy as np

In [2]:
ee.Authenticate()

Enter verification code: 4/1AbUR2VOgDfbAfrAw20Wytiq1EZ6ZbQfgFMiIRvcfsd-J1zxv6esWkOFANaE

Successfully saved authorization token.


In [ ]:
ee.Initialize()

import geetools

In [4]:
# Define an ImageCollection
site = ee.Geometry.Point([-72, -42]).buffer(1000)
collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(site).limit(5)
# Set parameters
bands = ['B2', 'B3', 'B4']
scale = 30
name_pattern = '{sat}_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d}'
date_pattern = 'ddMMMy'  # dd: day, MMM: month (JAN), y: year
folder = 'Earth_Images'
data_type = 'uint32'
extra = dict(sat='L8SR')
region = site
# Export
tasks = geetools.batch.Export.imagecollection.toDrive(
    collection=collection,
    folder=folder,
    region=region,
    namePattern=name_pattern,
    scale=scale,
    dataType=data_type,
    datePattern=date_pattern,
    extra=extra,
    verbose=True,
    maxPixels=int(1e13)
)

exporting L8SR_07Apr2013_232-89 to folder 'Earth_Images' in GDrive
exporting L8SR_21Apr2013_232-89 to folder 'Earth_Images' in GDrive
exporting L8SR_23May2013_232-89 to folder 'Earth_Images' in GDrive
exporting L8SR_08Jun2013_232-89 to folder 'Earth_Images' in GDrive
exporting L8SR_24Jun2013_232-89 to folder 'Earth_Images' in GDrive


# First fix

In [18]:
# Define an ImageCollection
site = ee.Geometry.Point([-70, -42]).buffer(1000)
collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(site).limit(5)

# Set parameters
bands = ['B4', 'B3', 'B2']  # These bands correspond to RGB
scale = 30
name_pattern = '{sat}_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d}'
date_pattern = 'ddMMMy'  # dd: day, MMM: month (JAN), y: year
folder = 'Earth_Images'
data_type = 'uint8'
extra = dict(sat='L8SR')
region = site

# Normalize and visualize images before exporting
collection = collection.map(lambda image: image.select(
    bands).visualize(min=0, max=10_000))  # Adjust min and max values based on your data

# Export
tasks = geetools.batch.Export.imagecollection.toDrive(
    collection=collection,
    folder=folder,
    region=region,
    namePattern=name_pattern,
    scale=scale,
    dataType=data_type,
    datePattern=date_pattern,
    extra=extra,
    verbose=True,
    maxPixels=int(1e13),
)

exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive
exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive
exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive
exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive
exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive


# Second Fix - Remove Clouds?

In [42]:
# Define an ImageCollection
site = ee.Geometry.Point([-104, 39]).buffer(4000)
collection = (ee.ImageCollection("COPERNICUS/S2_SR")
              .filterBounds(site)
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .filterDate('2020-06-01', '2020-07-30')
              .limit(4))

# Set parameters
bands = ['B4', 'B3', 'B2']  # These bands correspond to RGB
scale = 15
name_pattern = '{sat}_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d}'
date_pattern = 'ddMMMy'  # dd: day, MMM: month (JAN), y: year
folder = 'Earth_Images'
data_type = 'uint8'
extra = dict(sat='L8SR')
region = site


# Normalize and visualize images before exporting
collection = collection.map(lambda image: image.select(
    bands).visualize(min=0, max=10_000))  # Adjust min and max values based on your data

# Export
tasks = geetools.batch.Export.imagecollection.toDrive(
    collection=collection,
    folder=folder,
    region=region,
    namePattern=name_pattern,
    scale=scale,
    dataType=data_type,
    datePattern=date_pattern,
    extra=extra,
    verbose=True,
    maxPixels=int(1e13),
)

exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive
exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive
exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive
exporting L8SR_{system_date}_{WRS_PATH:%d}-{WRS_ROW:%d} to folder 'Earth_Images' in GDrive


In [105]:

# Variety of Configuations
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 10_000 #Visualization settings for the thumbnail
vis_bands = ['B4', 'B3', 'B2'] #Includes the bands for RGB 
imageDimensions = '1024x1024' #Set thumbnail image size (can't be too big, or you run into problems)

id_ = 1


#Choose your location to request an image from
longitude = -104
latitude = 40


center = ee.Geometry.Point(longitude,latitude)

# Import Sentinel dataset
s2 = (ee.ImageCollection("COPERNICUS/S2_SR")   # LANDSAT/LC08/C01/T1_SR or COPERNICUS/S2_SR
  .filterBounds(center)
  .sort('CLOUDY_PIXEL_PERCENTAGE',True)
  .filterDate('2020-06-01', '2020-07-30')
  .first()
)

try:
    global sentinel_footprint
    sentinel_footprint = (ee.Geometry.Polygon((s2.getInfo().get('properties').get('system:footprint').get('coordinates'))))
  #print(sentinel_footprint)
except:
    print("No Footprint found")
  #continue


# Create a rectangular export area
exportArea = (sentinel_footprint.centroid().buffer(10000).bounds())

# This part is nicer in the JavaScript version, but I did not get it running for Python yet

s2Vis = {
    'region': exportArea,
    'crs': (s2.select('B4').projection()),
    'dimensions': imageDimensions,
    'format': 'png',
    'min': vis_min,
    'max': vis_max,
    'bands': vis_bands,
    'scale': 15
}

s2_url = (s2.getThumbURL(s2Vis))


urllib.request.urlretrieve(s2_url, f"../raw_data/individual_photos/image{id_}_{longitude}*{latitude}.png")
print(s2_url)

EEException: Image.clipToBoundsAndScale: "scale" may not be specified along with "width", "height", or "maxDimension".

In [ ]:
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 10_000 #Visualization settings for the thumbnail
vis_bands = ['map'] #Includes the bands for RGB 
imageDimensions = '1024x1024' #Set thumbnail image size (can't be too big, or you run into problems)
start_location = 0 # where you will start your loop based on the dataframe


for index,row in locations.iloc[start_location:].iterrows():
    id_ = index
    longitude = row['longitude']
    latitude = row['latitude']

    center = ee.Geometry.Point(longitude,latitude)

    # Import Sentinel dataset
    s2 = (ee.ImageCollection("ESA/WorldCover/v100")   # LANDSAT/LC08/C01/T1_SR or COPERNICUS/S2_SR
      .filterBounds(center)
      .sort('CLOUDY_PIXEL_PERCENTAGE',True)
      .filterDate('2020-05-01', '2020-06-30')
      .first()
    )

    try:
        global sentinel_footprint
        sentinel_footprint = (ee.Geometry.Polygon((s2.getInfo().get('properties').get('system:footprint').get('coordinates'))))
    except:
        print("No Footprint found")
        continue


    # Create a rectangular export area
    exportArea = (sentinel_footprint.centroid().buffer(5_000).bounds())

    # This part is nicer in the JavaScript version, but I did not get it running for Python yet

    s2Vis = {
        'region': exportArea,
        'crs': (s2.select('B4').projection()),
        'dimensions': imageDimensions,
        'format': 'png',
        'min': vis_min,
        'max': vis_max,
        'bands': vis_bands,
        'gamma':1.5
    }

    s2_url = (s2.getThumbURL(s2Vis))


    urllib.request.urlretrieve(s2_url, f"../raw_data/ESA_worldcover/image{id_}_{longitude}*{latitude}.png")
    print(f'finished number {id_} at {round(longitude,2)} : {round(latitude,2)}')

No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint found
No Footprint

In [125]:
lat1 = 37.
lat2 = 41.

lon1 = -102.
lon2 = -109.
longitudes = []
latitudes = []

for i in np.arange(lon2,lon1,0.5):
    for j in np.arange(lat1,lat2,0.5):
        longitudes.append(i)
        latitudes.append(j)

In [126]:
locations = pd.DataFrame({
    'longitude': longitudes,
    'latitude': latitudes
})
locations

,longitude,latitude
0,-109.0,37.0
1,-109.0,37.5
2,-109.0,38.0
3,-109.0,38.5
4,-109.0,39.0
...,...,...
107,-102.5,38.5
108,-102.5,39.0
109,-102.5,39.5
110,-102.5,40.0
